Dorothy and the Wizard in OZ soruce: https://gutenberg.org/cache/epub/22566/pg22566.txt

In [2]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW
